# Generate Simulations

In [ ]:
from orix.crystal_map import Phase
from orix.sampling import get_sample_reduced_fundamental
from diffsims.generators.simulation_generator import SimulationGenerator

# cif_file = "/Users/xiaodong/Desktop/simulations/MFM300-VIII/4135627.cif"
cif_file = "/Users/xiaodong/Desktop/simulations/LTA/7108314.cif"
resolution = 10                  # angular resolution(granularity) of simulation in degrees
reciprocal_radius = 1/0.5       # 1/Å (i.e. Å⁻¹)
with_direct_beam = False        # Include the direct beam
max_excitation_error = 1        # approx 1/thickness of crystal
shape_factor_width = 0.01       # None or float if none then set to max_excitation_error
accelerating_voltage = 300      # kV

# Get phase from cif
phase = Phase.from_cif(cif_file)

# Sample orientations in the symmetry-reduced zone (resolution in degrees)
orientations = get_sample_reduced_fundamental(resolution=resolution, point_group=phase.point_group)

# Initialize the simulation generator (e.g., setting the accelerating voltage)
generator = SimulationGenerator(accelerating_voltage = accelerating_voltage, approximate_precession = False)

# Calculate 2D diffraction simulations using the provided phase and orientations
simulations = generator.calculate_diffraction2d(
    phase = phase,
    rotation = orientations,
    reciprocal_radius = reciprocal_radius,
    with_direct_beam = with_direct_beam,
    max_excitation_error = max_excitation_error,
    shape_factor_width = shape_factor_width,
    debye_waller_factors = None,
    show_progressbar = True
)

# simulations.plot();

In [ ]:
import os
from cif2cell_gemmi import extract_parameters, write_cell
params = extract_parameters(cif_file)

print('lattice type:',  params['lattice_type'])
print('centering:',     params['centering'])
print('bravais lattice:',   params['bravais'])
base, _ = os.path.splitext(cif_file)
cell_path = base + '.cell'
write_cell(params, cell_path)
print(f"Wrote: {cell_path}")

# Add Simulations to Image with or without Background

In [ ]:
import os
from tqdm import tqdm
import h5py
from cif2cell_gemmi import extract_parameters, write_cell
from generate_geom import write_geom_file
from calculate_calibration import calculate_calibration
from create_empty_backgrounds import create_empty_backgrounds
from helper_functions_UB import copy_h5_file, get_next_simulation_folder
from compute_B import compute_B

output_cell = os.path.basename(cif_file).replace(".cif", ".cell")
output_geom = os.path.basename(cif_file).replace(".cif", ".geom")

params = extract_parameters(cif_file)
write_cell(params, output_cell)

lattice = phase.structure.lattice
cell = (
    lattice.a,
    lattice.b,
    lattice.c,
    lattice.alpha,
    lattice.beta,
    lattice.gamma,
)
in_plane_angle = 180                # set to correspond to geometry file definitions of xs and ss, if p0/corner_x = -512.5 p0/corner_y = -512.5 set to 180
sigma = 2                           # spread of the  in pixels
wavelength = 0.019687               # wavelength in Å
clen = 0.3                          # camera length in m
pixels_per_m = 17857.14285714286    # pixels per meter (i.e. 1e-6 m/pixel)
write_geom_file(
    filename=output_geom,
    wavelength=wavelength,
    clen=clen,
    res=pixels_per_m
)
calibration = calculate_calibration(
    wavelength_A=wavelength, 
    clen_m=clen, res=pixels_per_m)  # calculated based on geometry (use script to calculate)
fast = False                        # use fast if no subpixel accuracy is needed
normalize = True                    # normalize the intensities of simulated spots
fast_clip_threshold = 1e-12         # threshold for clipping the fast simulation
intensity_scale = 10000             # scale the intensity of the simulated spots

working_dir = os.path.dirname(cif_file)
create_empty_backgrounds(working_dir, simulations.current_size)
input_filename = os.path.join(working_dir, f"{simulations.current_size}_empty_backgrounds.h5")

# Compute the B matrix from cell parameters
B = compute_B(cell)

# 1. Create a new simulation folder
base_dir = os.path.dirname(input_filename)
new_sim_folder = get_next_simulation_folder(base_dir)

# 2. Copy the original file to the new simulation folder
output_filename = os.path.join(new_sim_folder, "sim.h5")
copy_h5_file(input_filename, output_filename)

# 3. Process the simulation images.
with h5py.File(output_filename, "r+") as data:
    images = data["entry"]["data"]["images"]
    orientation_matrices = data["entry"]["data"].require_dataset(
        "simulation_orientation_matrices", 
        shape=(images.shape[0], 3, 3), 
        dtype=float
    )
    det_shift_x_mm = data["entry"]["data"].require_dataset(
    "det_shift_x_mm",
    shape=(images.shape[0],),
    dtype=float,
    fillvalue=0
    )
    det_shift_y_mm = data["entry"]["data"].require_dataset(
    "det_shift_y_mm",
    shape=(images.shape[0],),
    dtype=float,
    fillvalue=0
    )

    shape = images.shape[-2:]
    beam_pos = (shape[0] // 2, shape[1] // 2)

    for i in tqdm(range(images.shape[0]), desc="Processing images"):
        img = simulations.irot[i].get_diffraction_pattern(
            shape=shape,
            direct_beam_position=beam_pos,
            in_plane_angle=in_plane_angle,  
            sigma=sigma,
            calibration=calibration,
            fast=fast,
            normalize=normalize,
            fast_clip_threshold=fast_clip_threshold
        )
        img = (img * intensity_scale).astype(images.dtype)
        images[i] += img
        orientation_matrices_i = simulations.rotations[i].to_matrix().squeeze()
        
        orientation_matrices[i] = B @ orientation_matrices_i 

print("Processing complete. Updated file saved at:", output_filename)

# Construct the full file path
simulation_parameter_txt = os.path.join(new_sim_folder, "simulation_parameters.txt")

# Write variables and their values to the file
with open(simulation_parameter_txt, "w") as file:
    file.write(f"cif_file = {cif_file}\n")
    file.write(f"resolution = {resolution}\n")
    file.write(f"reciprocal_radius = {reciprocal_radius}\n")
    file.write(f"with_direct_beam = {with_direct_beam}\n")
    file.write(f"max_excitation_error = {max_excitation_error}\n")
    file.write(f"shape_factor_width = {shape_factor_width}\n")
    file.write(f"accelerating_voltage = {accelerating_voltage}\n")
    file.write(f"cell = {cell}\n")
    file.write(f"input_filename = {input_filename}\n")
    file.write(f"shape = {shape}\n")
    file.write(f"beam_pos = {beam_pos} (actually 0.5 pixels less i.e use p0/corner_x, p0/corner_y = -{(beam_pos[0] + 0.5, beam_pos[1] + 0.5)} in .geom file)\n")
    file.write(f"sigma = {sigma}\n")
    file.write(f"calibration = {calibration}\n")
    file.write(f"fast = {fast}\n")
    file.write(f"normalize = {normalize}\n")
    file.write(f"fast_clip_threshold = {fast_clip_threshold}\n")
    file.write(f"intensity_scale = {intensity_scale}\n")

print(f"Variables have been written to {simulation_parameter_txt}")

# Read back the entire dataset into memory
with h5py.File(output_filename, "r") as data:
    orientation_matrices_np = data["entry"]["data"]["simulation_orientation_matrices"][:]
output_sol = os.path.join(new_sim_folder, "orientation_matrices.sol")

# Write orientation matrices to a .sol file
with open(output_sol, "w") as sol_file:
    for i, matrix in enumerate(orientation_matrices_np):
        # Flatten the 3x3 matrix (row-major order)
        # Each number is formatted with a sign (+ or -) and 7 decimal places.
        line = " ".join(f"{num:+.7f}" for num in matrix.flatten())
        sol_file.write(f"{output_filename} //{i} " + line + f" 0.000 0.000 {params['bravais']}\n")

# Show Simulations on Background from .h5 file

In [ ]:
# To make sure plots display inline
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact
from helper_functions_UB import load_h5_data, view_image

# 4. Load the processed data for interactive visualization
images_arr, orientation_matrices_str = load_h5_data(output_filename)

# 5. Set up the interactive viewer
interact(lambda index: view_image(index, images_arr, orientation_matrices_str), 
         index=widgets.IntSlider(min=0, max=images_arr.shape[0]-1, step=1, value=0))

# Integration from .sol file

In [ ]:
from gandalf_iterator import gandalf_iterator

geomfile_path = "/Users/xiaodong/Desktop/simulations/LTA/LTAsim.geom"
cellfile_path = "/Users/xiaodong/Desktop/simulations/LTA/LTA.cell"
input_path =   "/Users/xiaodong/Desktop/simulations/LTA/simulation-6"
input_sol_file = "/Users/xiaodong/Desktop/simulations/LTA/simulation-6/orientation_matrices.sol"

output_file_base = "from_file"

num_threads = 8
x, y = 512.5, 512.5
step_size = 0.5
layers = 0

extra_flags=[
# PEAKFINDING
"--no-revalidate",
"--no-half-pixel-shift",

# "--peaks=peakfinder9",
# "--min-snr=1",
# "--min-snr-biggest-pix=1",
# "--min-sig=10",
# "--local-bg-radius=5",

"--peaks=peakfinder8",
"--threshold=1",
"--min-snr=1",
"--min-pix-count=1",
"--max-pix-count=500",
"--local-bg-radius=10",
"--min-res=1",
"--max-res=800",

# INDEXING
"--indexing=file",
f"--fromfile-input-file={input_sol_file}",
"--no-check-cell",
"--no-check-peaks",
"--no-retry",
"--no-refine",
# INTEGRATION
"--integration=rings",
"--int-radius=2,5,10",
# OUTPUT
"--no-non-hits-in-stream",
"--fix-profile-radius=40000000",
]

"""Examples of extra flags(see crystfel documentation https://www.desy.de/~twhite/crystfel/manual-indexamajig.html):"""

""" Basic options
"--highres=n",
"--no-image-data",
"""

""" Peakfinding
"--peaks=cxi",
"--peak-radius=inner,middle,outer",
"--min-peaks=n",
"--median-filter=n",
"--filter-noise",
"--no-revalidate",
"--no-half-pixel-shift",

"--peaks=peakfinder9",
"--min-snr=1",
"--min-snr-peak-pix=6",
"--min-snr-biggest-pix=1",
"--min-sig=9",
"--min-peak-over-neighbour=5",
"--local-bg-radius=5",

"--peaks=peakfinder8",
"--threshold=45",
"--min-snr=3",
"--min-pix-count=3",
"--max-pix-count=500",
"--local-bg-radius=9",
"--min-res=30",
"--max-res=500",
"""

""" Indexing
"--indexing=xgandalf",

"--tolerance=tol"
"--no-check-cell",
"--no-check-peaks",
"--multi",
"--no-retry",
"--no-refine",

"--xgandalf-sampling-pitch=n"
"--xgandalf-grad-desc-iterations=n"
"--xgandalf-tolerance=n"
"--xgandalf-no-deviation-from-provided-cell"
"--xgandalf-max-lattice-vector-length=n"
"--xgandalf-min-lattice-vector-length=n"
"--xgandalf-max-peaks=n"
"--xgandalf-fast-execution"
"""

""" Integration
"--integration=rings",
"--int-radius=10,11,15",
"--push-res=n",
"--overpredict",
"--cell-parameters-only",
"""

""" Output
"--no-non-hits-in-stream",
"--no-peaks-in-stream",
"--no-refls-in-stream",
"--serial-offset
"""

gandalf_iterator(x, y, geomfile_path, cellfile_path, input_path, output_file_base, num_threads, step_size, layers, extra_flags=extra_flags)


# Index using XGANDALF and Integration using rings

In [ ]:
from gandalf_iterator import gandalf_iterator

geomfile_path = "/Users/xiaodong/Desktop/simulations/LTA/LTAsim.geom"
cellfile_path = "/Users/xiaodong/Desktop/simulations/LTA/LTA.cell"
input_path =   "/home/bubl3932/files/LTA_sim/simulation-42"
output_file_base = "LTA"

num_threads = 24
x, y = 512.5, 512.5
step_size = 0.01
layers = 0
extra_flags=[
# PEAKFINDING
"--no-revalidate",
"--no-half-pixel-shift",
"--peaks=peakfinder9",
"--min-snr=1",
"--min-snr-biggest-pix=1",
"--min-sig=3",
"--local-bg-radius=10",
# INDEXING
"--indexing=xgandalf",
# "--tolerance=10,10,10,5",
"--no-refine",
"--xgandalf-sampling-pitch=5",
"--xgandalf-grad-desc-iterations=1",
"--xgandalf-tolerance=0.02",
"--xgandalf-no-deviation-from-provided-cell",
# INTEGRATION
"--integration=rings",
"--int-radius=4,5,9",
"--fix-profile-radius=70000000",
# OUTPUT
"--no-non-hits-in-stream",
]

"""Examples of extra flags(see crystfel documentation https://www.desy.de/~twhite/crystfel/manual-indexamajig.html):"""

""" Basic options
"--highres=n",
"--no-image-data",
"""

""" Peakfinding
"--peaks=cxi",
"--peak-radius=inner,middle,outer",
"--min-peaks=n",
"--median-filter=n",
"--filter-noise",
"--no-revalidate",
"--no-half-pixel-shift",

"--peaks=peakfinder9",
"--min-snr=1",
"--min-snr-peak-pix=6",
"--min-snr-biggest-pix=1",
"--min-sig=9",
"--min-peak-over-neighbour=5",
"--local-bg-radius=5",

"--peaks=peakfinder8",
"--threshold=45",
"--min-snr=3",
"--min-pix-count=3",
"--max-pix-count=500",
"--local-bg-radius=9",
"--min-res=30",
"--max-res=500",
"""

""" Indexing
"--indexing=xgandalf",

"--tolerance=tol"
"--no-check-cell",
"--no-check-peaks",
"--multi",
"--no-retry",
"--no-refine",

"--xgandalf-sampling-pitch=n"
"--xgandalf-grad-desc-iterations=n"
"--xgandalf-tolerance=n"
"--xgandalf-no-deviation-from-provided-cell"
"--xgandalf-max-lattice-vector-length=n"
"--xgandalf-min-lattice-vector-length=n"
"--xgandalf-max-peaks=n"
"--xgandalf-fast-execution"
"""

""" Integration
"--fix-profile-radius=n",
"--fix-divergence=n",
"--integration=rings",
"--int-radius=4,5,10",
"--push-res=n",
"--overpredict",
"--cell-parameters-only",
"""

""" Output
"--no-non-hits-in-stream",
"--no-peaks-in-stream",
"--no-refls-in-stream",
"--serial-offset
"""

gandalf_iterator(x, y, geomfile_path, cellfile_path, input_path, output_file_base, num_threads, step_size, layers, extra_flags=extra_flags)


In [ ]:
import os 
from run_partialator_and_convert import run_partialator_and_convert
from convert_hkl_crystfel_to_shelx import convert_hkl_crystfel_to_shelx

stream_file = "/home/bubl3932/files/LTA_sim/simulation-41/LTAsim_from_file_-512.5_-512.5.stream"
#"/Users/xiaodong/Desktop/simulations/LTA/simulation-43/LTAsim_from_file_-512.5_-512.5.stream" # Stream file to be merged
pointgroup = "m-3m"  # Point group of the crystal
num_threads = 24
iterations = 5

output_dir = run_partialator_and_convert(
    stream_file,
    pointgroup=pointgroup,
    num_threads=num_threads,
    iterations=iterations,
)

if output_dir is not None:
    print("All done. Results are in:", output_dir)

convert_hkl_crystfel_to_shelx(os.path.join(output_dir, "crystfel.hkl"))
